In [2]:
import cv2
import numpy as np
from insightface.app import FaceAnalysis

In [4]:
app_sc = FaceAnalysis(name='buffalo_sc',providers=['CPUExecutionProvider'])
app_sc.prepare(ctx_id=0,det_size=(640,640))

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\umesh/.insightface\models\buffalo_sc\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\umesh/.insightface\models\buffalo_sc\w600k_mbf.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


Step 1 : Collect Person data and role

In [18]:
person_name = input("Enter your name : ")
retry = 3
for i in range(retry):
    role= input("""
    Please choose role
    1. Student
    2. Teacher
    Enter 1 for Teacher and 2 for Student
    """)
    if role in ('1','2'):
        if role == '1':
            role = 'Student'
        else:
            role == 'Teacher'
            
        break   
    else:
        print("Invalid entry")
        retry -= 1
else:
    print("Maximum Attempts exceeded")

Enter your name : Ram

    Please choose role
    1. Student
    2. Teacher
    Enter 1 for Teacher and 2 for Student
    1


In [19]:
print(f"Name : {person_name} and Role:{role}")

Name : Ram and Role: Student


Step 2: Collect Samples

In [21]:
# define a video capture object 
vid = cv2.VideoCapture(0) 
  
while(True): 
      
    # Capture the video frame 
    # by frame 
    ret, frame = vid.read() 
  
    # Display the resulting frame 
    cv2.imshow('frame', frame) 
      
    # the 'q' button is set as the 
    # quitting button you may use any 
    # desired button of your choice 
    if cv2.waitKey(1) == ord('q'): 
        break
  
# After the loop release the cap object 
vid.release() 
# Destroy all the windows 
cv2.destroyAllWindows() q

Step 3: Optimize Data and save in Redis database